In [ ]:
%%time
import malaya
import tweepy
import pandas as pd
import re   
from wordcloud import WordCloud
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
consumer_key = TWITTER_API_CONSUMER_KEY
consumer_secret = TWITTER_API_CONSUMER_SECRET
access_token = TWITTER_API_ACCESS_TOKEN
access_token_secret = TWITTER_API_TOKEN_SECRET

auth=tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)

In [ ]:
api=tweepy.API(auth)

In [ ]:
# regex_txt = "https"
# regex = re.compile(regex_txt, re.IGNORECASE)
# match = regex.search(regex_txt)  # From your file reading code.
# if match is not None:
#     break
q="brgsjks"

In [ ]:
public_tweets=api.search_tweets(q+' exclude:retweets exclude:replies',count='100',tweet_mode="extended")
i=1
for tweet in public_tweets:
    print(str(i)+')'+tweet.full_text+" \n")
    i=i+1

In [ ]:
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame([tweet.full_text for tweet in public_tweets], columns=['Tweets'])

# Show 5 rows of data
df.head()

In [ ]:
# filter columns
def cleanTweet(tweet):
    tweet= re.sub(r'https?:\/\/\S+','',tweet) #remove hyperlinks
    tweet=re.sub(r'&[A-Za-z0-9!@#$%^&*()]','', tweet) 
    return tweet

df['Tweets']=df['Tweets'].apply(cleanTweet)
df['name_length'] = df['Tweets'].str.len()
df.head()

In [ ]:
# df = df[df['name_length'] > 1]    
df=df.drop(columns=['name_length'])
df.head()

In [ ]:
model = malaya.sentiment.multinomial()

In [ ]:
# MALAYA MULTINOMIAL MODEL
def getMultinomialSentiment(tweet):
    return model.predict([tweet])

df['Sentiment_1']= df['Tweets'].apply(getMultinomialSentiment)
# Show new data
df.head()

In [ ]:
# MALAYA MULTINOMIAL MODEL PROBABILITY
def getMultinomialProba(tweet):
    return model.predict_proba([tweet])

df['Probability']= df['Tweets'].apply(getMultinomialProba)
# Show new data
print(df['Probability'])                                                            

In [ ]:
quantized_model = malaya.sentiment.transformer(model = 'fastformer', quantized = True)


In [ ]:
# MALAYA MULTINOMIAL MODEL
def getFastSentiment(tweet):
    return model.predict([tweet])

df['Sentiment_2']= df['Tweets'].apply(getFastSentiment)
# Show new data
df.head()

In [ ]:
df['Tweets'].shape

In [ ]:
print(df.values)

In [ ]:
df.to_csv(r'tweet_malaya_brgsjks.csv', index = False)

In [ ]:
#Plot the World CLoud
allWords= ' '.join([twts for twts in df['Tweets']])
wordCloud = WordCloud(width=500,height=300, random_state=21, max_font_size=120).generate(allWords)

plt.imshow(wordCloud, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
COLOR = 'black'
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR

In [ ]:
df['Sentiment_2'].value_counts()

plt.title("Sentiment Analysis")
plt.xlabel("Sentiment")
plt.ylabel("Count")
df['Sentiment_2'].value_counts().plot(kind='bar')
plt.show()
